In [2]:
from quantdev.data import Databank
from quantdev.backtest import *
from quantdev.analysis import *

#### Classic

In [101]:
ROE = get_factor('常續ROE')
MTM = get_factor('mtm_3m')
PBR = get_factor('股價淨值比', asc=False)
classic_tw3factor = backtesting(
    get_factor(ROE+PBR+MTM)>=0.99, # 等權三因子, 並選前 1% 投資
    rebalance='Q' # 每一季底再平衡 (使用最近一期的財報資料)
)

,Strategy,0050.TT
total_return,12825.82%,711.82%
annual_return,27.29%,10.67%
mdd,-60.41%,-55.66%
annual_vol,20.21%,19.20%
calmar_ratio,0.45,0.19
sharpe_ratio,1.4,0.56
beta,0.61,-


In [26]:
classic_tw3factor._plot_equity_curve()

#### Profit

In [67]:
profit_factors = {
    'ROE': get_factor('常續ROE'),
    'ROE_Y': get_data('常續ROE_yoy_diff'),
    'ROE_Q': get_data('常續ROE_qoq_diff'),
}

profit_quantiles = pd.DataFrame()
for k, v in profit_factors.items():
    profit_quantiles = pd.concat([
        profit_quantiles, 
        factor_analysis(v, rebalance='QR').summary_dfs['quantiles'].rename(columns={'Quantiles Returns':k})
        ], axis=1)
display(profit_quantiles)

,ROE,ROE_Y,ROE_Q
0,1.42%,-2.51%,3.07%
1,6.83%,3.64%,6.56%
2,9.54%,6.92%,9.49%
3,9.95%,8.67%,10.22%
4,5.98%,11.11%,11.24%
5,14.13%,13.68%,15.21%
6,14.14%,14.57%,16.21%
7,14.90%,16.78%,16.57%
8,17.13%,18.66%,19.14%
9,18.26%,20.50%,17.24%


#### Value

In [77]:
value_factors = {
    'PER':get_factor('本益比', asc=False),
    'PBR':get_factor('股價淨值比',asc=False),
    'PSR':get_factor(get_data('收盤價')/get_data('每股營業收入'),asc=False),
    'MCAP_TO_REV':get_factor(get_data('個股市值(元)')/get_data('營業收入'), asc=False),
}


value_quantiles = pd.DataFrame()
for k, v in value_factors.items():
    value_quantiles = pd.concat([
        value_quantiles, 
        factor_analysis(v, rebalance='Q').summary_dfs['quantiles'].rename(columns={'Quantiles Returns':k})
        ], axis=1)

display(value_quantiles)

,PER,PBR,PSR,MCAP_TO_REV
0,7.51%,3.96%,5.59%,5.88%
1,10.12%,7.20%,7.38%,7.26%
2,9.19%,8.96%,10.97%,10.63%
3,9.93%,9.40%,9.93%,9.93%
4,11.63%,11.79%,11.10%,11.00%
5,13.74%,11.93%,12.82%,12.91%
6,13.60%,12.02%,13.34%,13.42%
7,13.88%,14.43%,13.47%,13.50%
8,14.44%,17.73%,13.48%,13.25%
9,9.86%,17.85%,17.43%,17.82%


#### Momentum

In [85]:
returns = get_data('報酬率')
turnover = get_data('周轉率')

days = 20
FIP = get_factor(
    get_factor(
        returns.apply(lambda x: ((x>0).rolling(days).sum()))
    ) + 
    get_factor(turnover.rolling(days).mean(), asc=False)
)

In [86]:
MTM = get_factor('mtm_3m')

In [87]:
mtm_quantiles = pd.concat(
    [
        factor_analysis(MTM, rebalance='M').summary_dfs['quantiles'].rename(columns={'Quantiles Returns':'MTM'}),
        factor_analysis(FIP, rebalance='M').summary_dfs['quantiles'].rename(columns={'Quantiles Returns':'FIP'}),
    ],
    axis=1
)
display(mtm_quantiles)

,MTM,FIP
0,8.56%,1.38%
1,6.91%,4.58%
2,7.25%,8.57%
3,9.34%,11.08%
4,11.48%,11.33%
5,11.92%,12.23%
6,13.98%,14.93%
7,14.88%,13.53%
8,15.73%,15.04%
9,11.44%,15.54%


#### Parameters

In [3]:
ROE_Y = get_factor('常續ROE_yoy_diff')
MCAP_TO_REV = get_factor(get_data('個股市值(元)')/get_data('營業收入'), asc=False)

returns = get_data('報酬率')
turnover = get_data('周轉率')
days = 20
FIP = get_factor(
    get_factor(
        returns.apply(lambda x: ((x>0).rolling(days).sum()))
    ) + 
    get_factor(turnover.rolling(days).mean(), asc=False)
)

In [102]:
improved_tw3factor = backtesting(
   get_factor(ROE_Y+MCAP_TO_REV+FIP)>=0.99, # 等權三因子
   rebalance='Q',
)

,Strategy,0050.TT
total_return,45830.07%,711.82%
annual_return,37.79%,10.67%
mdd,-47.40%,-55.66%
annual_vol,16.78%,19.20%
calmar_ratio,0.8,0.19
sharpe_ratio,2.3,0.56
beta,0.41,-


In [6]:
close = get_data('收盤價')
above_20d_ma = close>=close.rolling(20).mean()

In [98]:
improved_tw3factor2 = backtesting(
   get_factor(ROE_Y+MCAP_TO_REV+FIP, universe=above_20d_ma)>=0.99, 
   rebalance='MR', 
)

,Strategy,0050.TT
total_return,96718.19%,711.82%
annual_return,42.87%,10.67%
mdd,-34.05%,-55.66%
annual_vol,21.45%,19.20%
calmar_ratio,1.3,0.19
sharpe_ratio,2.0,0.56
beta,0.38,-


In [100]:
improved_tw3factor2._plot_equity_curve()